# Predicting Heart Disease

Cardiovascular diseases (CVDs) remain a leading cause of mortality globally and the [World Health Organization](https://www.who.int/health-topics/cardiovascular-diseases) (WHO) estimates that 17.9 million people die from CVDs every year. The WHO considers CVDs to be a serious public health challenge and recommends early identification of risk factors such as a unhealthy diet, lack of physical activity, or mental illness. Identifying these risk factors early on could help prevent many premature deaths.

![](https://www.nhlbi.nih.gov/sites/default/files/styles/meta_image/public/2022-09/Heart.jpg?itok=Mfe23Jii)

Image Source: [National Heart, Lung, and Blood Institute](https://www.nhlbi.nih.gov/news/2022/genetic-paths-predicting-heart-disease)

## Table of Contents
---
- [Introduction](#Introduction)
- [Importing Libraries and Reading Data](#Importing-Libraries-and-Reading-Data)
- [EDA: Descriptive Statistics](#EDA:-Descriptive-Statistics)
- [EDA: Visualizations](#EDA:-Visualizations)
- [Data Cleaning](#Data-Cleaning)
- [Feature Selection](#Feature-Selection)
- [Building a Classifier with One Feature](#Building-a-Classifier-with-One-Feature)
- [Building a Classifier with Multiple Features](#Building-a-Classifier-with-Multiple-Features)
- [Hyperparameter Tuning](#Hyperparameter-Tuning)
- [Model Evaluation on Test Set](#Model-Evaluation-on-Test-Set)
- [Conclusions](#Conclusions)

## Introduction

Our R&D company specializes in healthcare solutions and has collected anonymized data from multiple hospitals on several patients to predict the likelihood of a new patient having heart disease in the future. The [dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) includes relevant information for each patient, such as personal information and medical data, including whether or not they have had heart disease before.

The dataset has the following features:

- `Age`: age of the patient [years]
- `Sex`: sex of the patient [M: Male, F: Female]
- `ChestPainType`: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
- `RestingBP`: resting blood pressure [mm Hg]
- `Cholesterol`: serum cholesterol [mm/dl]
- `FastingBS`: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
- `RestingECG`: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
- `MaxHR`: maximum heart rate achieved [Numeric value between 60 and 202]
- `ExerciseAngina`: exercise-induced angina [Y: Yes, N: No]
- `Oldpeak`: oldpeak = ST [Numeric value measured in depression]
- `ST_Slope`: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
- `HeartDisease`: output class [1: heart disease, 0: Normal]

To accurately predict the likelihood of a new patient having heart disease in the future, we will conduct exploratory data analysis on the dataset before building a model.

## Importing Libraries and Reading Data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Set the maximum column width
pd.set_option('display.max_colwidth', 100)
# Display plots in notebook
%matplotlib inline

In [ ]:
# Read dataset and display first five rows
heart_df = pd.read_csv('heart_disease_prediction.csv')
heart_df.head()

In [ ]:
# Display number of features and observations
print(f'Shape of dataframe: {heart_df.shape}')
print(f'Number of observations: {heart_df.shape[0]}')
print(f'Number of features: {heart_df.shape[1]}')

In [ ]:
# Check the data types of the features
heart_df.dtypes

We will now perform Exploratory Data Analysis (EDA).

## EDA: Descriptive Statistics

After examining the list of features and inspecting some of the rows from the dataset, we have identified both categorical and numerical features. The categorical features have a data type of object in this dataset. Notably, two columns - `FastingBS` and `HeartDisease` - have a data type of **int64**, but are categorical variables since they only take the values `0` and `1`.

Our initial focus will be on investigating and analyzing the numerical columns.

In [ ]:
heart_df.describe()

## EDA: Visualizations

From the table above, we can observe that:

- The average age of patients is about `54` years.
- The median for `Cholesterol` is higher than its mean by roughly `25` mm/dl, indicating that it could be a left-skewed distribution with a possibility of outliers skewing the distribution.
- `RestingBP` and `Cholesterol` have a minimum value of zero.
- There don't seem to be any missing values in these columns. But we will have to confirm it across the entire dataset as well.

Two observations stand out. First, `RestingBP` and `Cholesterol` having a minimum value of zero seems unlikely. `RestingBP` is the resting blood pressure, and it's unlikely to be zero, while a cholesterol value of zero is also not possible as serum cholesterol is always present in the blood.

Secondly, we need domain expertise to help answer questions like this. For example, we could reach out to a subject matter expert or conduct further research on our own. As per the [American Heart Association](https://www.heart.org/en/health-topics/cholesterol/about-cholesterol/what-your-cholesterol-levels-mean), serum cholesterol:

> "is a composite of different measurements. Your total blood cholesterol is calculated by adding your HDL and LDL cholesterol levels, plus 20% of your triglyceride level."

Although these insights do not directly help us with our prediction model, they do provide us with valuable information regarding the quality of our dataset.

We will now move on to explore the categorical features. We will create visualizations to understand how the data is distributed in relation to our target variable -- `HeartDisease`. For example, we will look at how many patients had a resting ECG categorized as ST and how many female patients had heart disease.

In [ ]:
# Number of missing values in all columns
heart_df.isna().sum()

Thankfully, none of our columns have missing values.

We identified that most of the categorical columns are all of dtype **object**.

In [ ]:
heart_df.describe(include=['object'])

We can confirm that those columns are indeed categorical given the number of unique values in each of them. But, we can't gather much else. Also, `FastingBS` and `HeartDisease` are categorical as well since they only contain binary values. We can confirm that quickly as well.

In [ ]:
heart_df["FastingBS"].unique(), heart_df["HeartDisease"].unique() 

We'll create a bar chart for each categorical column in the `heart_df` dataset, including `FastingBS` and `HeartDisease`. These charts will display the number of rows for each category in the column, with appropriate axis labels, data labels, and a title to make the visualization clear and informative.

In [ ]:
categorical_cols = ["Sex", "ChestPainType", "FastingBS", "RestingECG", "ExerciseAngina", "ST_Slope", "HeartDisease"]

fig = plt.figure(figsize=(16,15))

for idx, col in enumerate(categorical_cols):
    ax = plt.subplot(4, 2, idx+1)
    sns.countplot(x=heart_df[col], ax=ax)
    # add data labels to each bar
    for container in ax.containers:
        ax.bar_label(container, label_type="center")

Here are some relevant insights that can be gathered from the plot of each categorical column:

- The dataset is highly skewed towards male patients. There are `725` male patients and `193` female patients. This could potentially induce a bias in our model.
- `496` patients had `ASY` (asymptotic) chest pain type.
- `552` patients had a normal restin `ECG` (electrocardiogram).
- `704` patients had blood sugar lower than 120 mg/dl

Now our next step is to create bar charts for every categorical column in the dataset, including `FastingBS`. These bar charts will display the number of rows for each category of that column, grouped by `HeartDisease`.

In [ ]:
fig = plt.figure(figsize=(16,15))

for idx, col in enumerate(categorical_cols[:-1]):
    ax = plt.subplot(4, 2, idx+1)
    # group by HeartDisease
    sns.countplot(x=heart_df[col], hue=heart_df["HeartDisease"], ax=ax)
    # add data labels to each bar
    for container in ax.containers:
        ax.bar_label(container, label_type="center")

- We can further notice how skewed the dataset is towards male patients. Only `50` female patients in the dataset have been diagnosed with heart disease.
- A significant number of patients, `392`, diagnosed with heart disease have asymptomatic (ASY) chest pain. While chest pain could be a relevant feature for our model, asymptomatic implies that those patients who had a heart disease did not have chest pain as a symptom.
- A high number (`170`) of patients with blood sugar greater than 120 mg/dl were diagnosed with heart disease in relation to those who were not diagnosed as such.
- Out of all patients who had an exercise-induced angina, `316` were diagnosed with a heart disease.
- Out of all patients with a flat ST slope, `381` were diagnosed with a heart disease.

We will now answer the question: `What percentage of patients with heart disease are male and over 50 years old?`

In [ ]:
# Filter for patients with heart disease
heart_disease_df = heart_df[heart_df['HeartDisease'] == 1]

# Calculate percentage of patients with heart disease who are male and over 50 years old
male_over_50 = ((heart_disease_df['Sex'] == 'M') & (heart_disease_df['Age'] > 50)).sum()
total_heart_disease = len(heart_disease_df)
percent_male_over_50 = male_over_50 / total_heart_disease * 100

# Create pie chart
labels = ['Male and over 50', 'Other']
sizes = [percent_male_over_50, 100 - percent_male_over_50]
explode = (0.1, 0)

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', startangle=90)
ax.axis('equal')  # equal aspect ratio ensures that pie is drawn as a circle
ax.set_title('Percentage of Heart Disease Patients Who are Male and Over 50')
plt.show()

The graph above shows that approximately `68%` of the patients with heart disease are male and over 50 years old. This is a significant finding as it indicates that there may be a higher risk of developing heart disease in males over 50 years old. This information could be valuable for developing targeted prevention and treatment strategies for this population.

Next, we will answer the question: `What is the median age of patients who were diagnosed with heart disease?`

In [ ]:
# Set figure size
plt.figure(figsize=(8, 6))

# Create box plot
bp = plt.boxplot(x=[heart_df[heart_df['HeartDisease'] == 0]['Age'], heart_df[heart_df['HeartDisease'] == 1]['Age']],
                 labels=['No Heart Disease', 'Heart Disease'], patch_artist=True, widths=0.5, showfliers=True,
                 flierprops={'marker':'o', 'markerfacecolor':'black', 'markersize':5})

# Set boxplot colors
colors = ['#1f77b4', '#ff7f0e']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

# Set median line color
for median in bp['medians']:
    median.set(color='black', linewidth=2)

# Set axis labels and title
plt.xlabel('Heart Disease Diagnosis')
plt.ylabel('Age')
plt.title('Distribution of Age by Heart Disease Diagnosis')

# Set y-axis range
plt.ylim(20, 90)

# Add horizontal grid lines
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

Based on the boxplot visualization, we can see that the median age of patients without heart disease is `51` years old, while the median age of patients with heart disease is `57` years old. This suggests that age may be a risk factor for developing heart disease, as the median age of those with heart disease is higher than those without. However, it's important to note that there is overlap in the age distributions of those with and without heart disease, indicating that age is not the only factor influencing the development of heart disease.

## Data Cleaning

We identified that there are no missing values. However, as we noticed earlier, a couple of columns have numerical values of `0` which don't make sense.

We will look at how many zero values `RestingBP` and `Cholesterol` contain and decide how to fix them.

In [ ]:
heart_df[heart_df["RestingBP"] == 0]

In [ ]:
heart_df[heart_df["Cholesterol"] == 0]

`RestingBP` has only one zero value. We will remove that row. There are `172` zero values for `Cholesterol`. That's a relatively high number. We can't remove them all so instead we will replace them with the median.

To be more accurate, we will replace the zero values in `Cholesterol` in relation to `HeartDisease`. So, the numerical values of `0` in `Cholesterol` for patients who were diagnosed with a heart disease will be replaced by the median of the non-zero values for patients who were diagnosed with heart disase. We will also do the same for those who were not diagnosed with heart disease.

In [ ]:
heart_df_clean = heart_df.copy()

# Create a clean copy of RestingBP with only non-zero values
heart_df_clean = heart_df_clean[heart_df_clean["RestingBP"] != 0]

heart_disease_mask = heart_df_clean["HeartDisease"]==0

cholesterol_without_heart_disease = heart_df_clean.loc[heart_disease_mask, "Cholesterol"]
cholesterol_with_heart_disease = heart_df_clean.loc[~heart_disease_mask, "Cholesterol"]

# Update the clean df with the median of patients with and without heart disease 
heart_df_clean.loc[heart_disease_mask, "Cholesterol"] = cholesterol_without_heart_disease.replace(to_replace = 0, value = cholesterol_without_heart_disease.median())
heart_df_clean.loc[~heart_disease_mask, "Cholesterol"] = cholesterol_with_heart_disease.replace(to_replace = 0, value = cholesterol_with_heart_disease.median())
heart_df_clean[["Cholesterol", "RestingBP"]].describe()

We have now updated the minimal values for `Cholesterol` and `RestingBP`.

## Feature Selection

Now that we've cleaned our data, we can select features for our model. Thanks to our EDA and a general understanding of the features, we can identify some of the features that we could start with:

- `Age`
- `Sex`
- `ChestPainType`
- `Cholesterol`
- `FastingBS`

We will also identify how strongly feature columns are correlated to the target colummn. This should help us narrow down on the important features.

In order to do that, we will first convert our categorical columns into dummy variables.

In [ ]:
heart_df_clean = pd.get_dummies(heart_df_clean, drop_first=True)
heart_df_clean.head()

We will now see how these columns are correlated.

In [ ]:
correlations = abs(heart_df_clean.corr())
plt.figure(figsize=(12,8))
sns.heatmap(correlations, annot=True, cmap="Blues")

In [ ]:
plt.figure(figsize=(12,8))
# Zone in on the columns that are reasonably correlated to "HeartDisease"
sns.heatmap(correlations[correlations > 0.35], annot=True, cmap="Blues")

From our correlation heatmap, we can identify the following features to be positively correlated (with a correlation coefficient greater than `0.35` to `HeartDisease`:

- `Oldpeak`
- `MaxHR`
- `ChestPainType_ATA`
- `ExerciseAngina_Y`
- `ST_Slope_Flat`
- `ST_Slope_Up`

The correlation coefficient threshold was chosen arbitrarily. Surprisingly, `Cholesterol` is not strongly correlated to `HeartDisease`. Therefore, we can ignore further analysis on `Cholesterol`.

Given everything we have attempted so far, we can narrow down our important features as:

- `Oldpeak`
    - We could also select `MaxHR` or `ChestPainType_ATA`.
- `Sex_M`
    - It has a relatively low value for the coefficient, but given what we observed in our EDA, let's also take it into account.
- `ExerciseAngina_Y`
- `ST_Slope_Flat`
- `ST_Slope_Up`

We will now use machine learning to build a classifier!

## Building a Classifier with One Feature

We will first split our dataset into a training set and a test set.

In [ ]:
# We want to find the features which are correlated to "HeartDisease"
X = heart_df_clean.drop(["HeartDisease"], axis=1)
y = heart_df_clean["HeartDisease"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state = 417)

features = [
    "Oldpeak",
    "Sex_M",
    "ExerciseAngina_Y",
    "ST_Slope_Flat",
    "ST_Slope_Up"
]

We will use a KNN classifier to evaluate performance.

In [ ]:
for feature in features:
    knn = KNeighborsClassifier(n_neighbors = 5)
    knn.fit(X_train[[feature]], y_train)
    accuracy = knn.score(X_val[[feature]], y_val)
    print(f"The k-NN classifier trained on {feature} and with k = 5 has an accuracy of {accuracy*100:.2f}%")

Our best model is the `ST_Slope_Up` feature with an accuracy of around `84%`. The `ST_Slope_Flat` feature is our second best model. These make sense given the data distributions we saw previously.

We will now train a model using all features.

## Building a Classifier with Multiple Features

Before training on all of the features, we need to **normalize** the data first. We will use scikit-learn's MinMaxScaler to scale the values between 0 and 1, and then train the model again.

In [ ]:
# Normalize between 0 and 1
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[features])
X_val_scaled = scaler.transform(X_val[features])

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_scaled, y_train)
accuracy = knn.score(X_val_scaled, y_val)
print(f"Accuracy: {accuracy*100:.2f}")

The model's accuracy increased to around `86%!` Let's see if we can improve this with hyperparameter tuning.

##  Hyperparameter Tuning

Let's first prepare the data.

In [ ]:
X = heart_df_clean.drop(["HeartDisease"], axis=1)
y = heart_df_clean["HeartDisease"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 417)

features = [
    "Oldpeak",
    "Sex_M",
    "ExerciseAngina_Y",
    "ST_Slope_Flat",
    "ST_Slope_Up"
]
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[features])

We will use grid search to explore the following range of values for a couple of hyperparameters:

- For `k` in the `KNeighborsClassifier` in scikit-learn we will use values in the range of `1` to `20`.
- For `weights`, we will try both `uniform` and `distance`.
- We will use two distance metrics - `minkwoski` and `manhattan`.

We note that `minkwoski` is the default metric for the `KNeighborsClassifier` in scikit-learn. So, it is useful to experiment with other metrics.

In [ ]:
grid_params = {"n_neighbors": range(1, 20),
               "weights": ["uniform", "distance"],
                "metric": ["minkowski", "manhattan"]
              }

knn = KNeighborsClassifier()
knn_grid = GridSearchCV(knn, grid_params, scoring='accuracy')
knn_grid.fit(X_train_scaled, y_train)

In [ ]:
knn_grid.best_score_*100, knn_grid.best_params_

Unfortunately, our best model had an accuracy of only `83%` with `19` n_neighbors, `uniform` weights, and `minkowski` as the distance metric.

As `GridSearchCV` employs a cross-validation approach, it is reasonable to assume that this performed a good estimate of how the model performs compared to our prior attempts.

We will now evaulate the model on the test set.

## Model Evaluation on Test Set

We need to first normalize our test set similar to how we scaled our training set.

In [ ]:
X_test_scaled = scaler.transform(X_test[features])
predictions = knn_grid.best_estimator_.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
print(f" Model Accuracy on test set: {accuracy*100:.2f}")

Our model got an accuracy of around `86%`, which is really good. This means that our model is likely to correctly predict whether a patient is at risk for a heart disease around `86%` of the time.

However, the accuracy being higher than the one before raises some flags. We should analyze how the data is distributed.

In [ ]:
print("Distribution of patients by their sex in the entire dataset")
print(X.Sex_M.value_counts())

print("\nDistribution of patients by their sex in the training dataset")
print(X_train.Sex_M.value_counts())

print("\nDistribution of patients by their sex in the test dataset")
print(X_test.Sex_M.value_counts())

We used `Sex` as one of our features for training the model.

- `X` had `724` males and `193` females.
- `X_train` had `615` males and `164` females.
- `X_test` had `109` males and `29` females.

We can see that the above datasets have a significantly higher number of male patients than female ones. We briefly mentioned previously that this could present a bias because of this imbalance in our dataset and we can see it potentially impacts our model. If the test dataset doesn't have that many female patients and the model was trained on a dataset with more male patients, then it is understandable it has better accuracy on the test set. Of course, there could be other factors contributing to this discrepancy.

## Conclusions

Based on the insights gained from this project, we have developed a predictive model for heart disease detection using the provided dataset. Our EDA process helped us identify and address some data cleaning issues such as imputation of zero values. After selecting relevant features through correlation analysis, we experimented with different classifiers and ultimately achieved a test accuracy of `86.23%` with KNN.

Our final model was trained using the following features:

- `Oldpeak`
- `MaxHR`
- `ChestPainType_ATA`
- `ExerciseAngina_Y`
- `ST_Slope_Flat`
- `ST_Slope_Up`

However, given the limitations of our data this accuracy might not be indicative of a well performing model. Moving forward, there are several steps we can take to improve our results, for example:

- Try out different features.
- Expand the grid search parameters to identify more optimal hyperparameters.
- Explore other algorithms that might perform better than k-NN.
- Try and collect more data.

Also, it is important to note some limitations and potential drawbacks before implementing this model in a real-world healthcare setting.

Pros of using this model in a real-world healthcare setting:

- Early identification of patients at risk of heart disease could lead to early intervention and prevention of heart disease.
- Automated detection of heart disease could lead to more efficient use of healthcare resources and improved patient outcomes.
- Machine learning models can analyze large amounts of data quickly, providing healthcare professionals with valuable insights into patient risk factors.

Cons of using this model in a real-world healthcare setting:

- The model is based on retrospective data, which may not accurately reflect the current population or demographic changes.
- The model's accuracy may be affected by differences in data collection across different hospitals and healthcare systems.
- The model's performance may degrade over time as patient populations and risk factors change.
- There may be ethical and legal considerations related to the use of machine learning models in healthcare decision-making.
